# ReAct Agent Text-to-SQL



In [8]:
!pip install google-generativeai

In [9]:
from google.colab import userdata
import os

google_api_key = userdata.get('GOOGLE_API_KEY')

In [10]:
import google.generativeai as genai

genai.configure(api_key=google_api_key)

## Database Creation

In [15]:
import sqlite3

def create_sample_database():
    """
    Create a simple SQLite database with a 'companies' table and a 'customers' table.
    Each company has an id, title, and description.
    Each customer has an id, name, revenue, and a foreign key company_id referencing the companies table.
    This function resets the database for a clean slate.
    """
    conn = sqlite3.connect("sample.db")
    cur = conn.cursor()

    # Enable foreign key support in SQLite.
    cur.execute("PRAGMA foreign_keys = ON;")

    # Create the companies table.
    cur.execute("""
        CREATE TABLE companies (
            id INTEGER PRIMARY KEY,
            title TEXT NOT NULL,
            description TEXT
        )
    """)

    # Create the customers table with a foreign key to companies.
    cur.execute("""
        CREATE TABLE customers (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            revenue REAL NOT NULL,
            company_id INTEGER,
            FOREIGN KEY (company_id) REFERENCES companies(id) ON DELETE SET NULL
        )
    """)

    # Insert sample data into companies.
    companies_data = [
        (1, 'Acme Corp', 'Leading provider of innovative solutions'),
        (2, 'Beta Inc', 'Specializes in data analytics and insights'),
        (3, 'Gamma LLC', 'Pioneer in cloud computing services')
    ]
    cur.executemany("INSERT INTO companies (id, title, description) VALUES (?, ?, ?)", companies_data)

    # Insert sample data into customers.
    customers_data = [
        (1, 'Alice', 1500.0, 1),
        (2, 'Bob', 2400.0, 2),
        (3, 'Charlie', 1800.0, 1),
        (4, 'Diana', 3200.0, 3),
        (5, 'Ethan', 2100.0, 2)
    ]
    cur.executemany("INSERT INTO customers (id, name, revenue, company_id) VALUES (?, ?, ?, ?)", customers_data)

    conn.commit()
    return conn


In [16]:
create_sample_database()

## ReAct Text-to-SQL Setting Up

In [17]:
system_prompt = f"""
You are a helpful assistant that translates natural language queries into SQL queries for the following SQLite database schema.

Table "companies":
  - id: INTEGER PRIMARY KEY
  - title: TEXT NOT NULL
  - description: TEXT

Table "customers":
  - id: INTEGER PRIMARY KEY
  - name: TEXT NOT NULL
  - revenue: REAL NOT NULL
  - company_id: INTEGER (Foreign key referencing companies(id))

Instructions:
  1. When you receive a text query that is relevant to this schema, generate the corresponding SQL query. Return only the SQL query with no additional commentary or characters like ``` or quotes e.g. SELECT * FROM COMPANIES.
  2. If you receive a text query that includes an error message, generate a corrected SQL query that fixes the error, again returning only the SQL query.
  3. If the query is irrelevant, do not output any extra text.
"""

In [18]:
model = genai.GenerativeModel(model_name="gemini-1.5-flash", system_instruction=system_prompt)

## Retrieval Augmented Generation

In [19]:
def generate_sql_query(nl_request, feedback=None):
    """
    Generate a SQL query from a natural language request.
    Optionally include feedback if previous attempts encountered errors.
    """
    prompt = f"Translate the following natural language request into SQL: '{nl_request}'."
    if feedback:
        prompt += f"\nFeedback: {feedback}"
    print(f"Sending prompt to model:\n{prompt}\n")
    response = model.generate_content(prompt)
    sql_query = response.text.strip()
    print(f"Generated SQL query:\n{sql_query}\n")
    return sql_query

In [20]:
def execute_query(conn, query):
    """
    Execute the given SQL query on the SQLite database and return the results or error message.
    """
    cur = conn.cursor()
    try:
        cur.execute(query)
        rows = cur.fetchall()
        conn.commit()  # Commit if the query modifies data.
        return rows, None
    except Exception as e:
        return None, str(e)

In [24]:
import time



def test():
    nl_request = input("Enter natural language query: ").strip()
    if not nl_request:
        print("No natural language query provided.")
        return

    conn = sqlite3.connect("sample.db")

    max_iterations = 5
    iteration = 0
    feedback = None
    final_result = None

    while iteration < max_iterations:
        iteration += 1
        print(f"--- Iteration {iteration} ---")
        sql_query = generate_sql_query(nl_request, feedback)

        if not sql_query:
            print("No corresponding SQL query for this query.")
            break

        result, error = execute_query(conn, sql_query)

        if error:
            print(f"SQL execution error: {error}")
            feedback = f"The query resulted in the following error: {error}. Please adjust the SQL query accordingly."
            time.sleep(1)
        else:
            if not result:
                print("No results found.")
            else:
                print("Query executed successfully. Results:")
                for row in result:
                    print(row)
            final_result = result
            break

    if final_result is None:
        print("Failed to generate a valid SQL query after several iterations.")

    conn.close()

In [27]:
test()

Enter natural language query: Two customers that hate each other
--- Iteration 1 ---
Sending prompt to model:
Translate the following natural language request into SQL: 'Two customers that hate each other'.

Generated SQL query:
-- No SQL query can be generated from this request as there is no information about customer relationships in the database schema.

No results found.
